In [2]:
import os
import pandas as pd
import numpy as np

from textblob import TextBlob


In [47]:
os.chdir('../scripts/')
import data_loader as data_loader

In [48]:
all_data_path = '../data/raw_analyst_ratings.csv'
df1 = data_loader.read_csv_file(all_data_path)

In [49]:
df1 = df1.get("data")
df1.head()

,headline,url,publisher,date,stock
0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A
1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A
2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A
3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A
4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A


In [50]:
df1['date'] = pd.to_datetime(df1['date'], format='ISO8601')
df1.head()

,headline,url,publisher,date,stock
0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A
1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A
2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A
3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A
4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A


# Correlation for Amazon

In [51]:
df1['stock'].unique()
grouped = df1.groupby('stock')

# Access the group for AMZN
amzn_group = grouped.get_group('AMZN')



In [52]:
all_data_path = '../data/yfinance_data/AMZN_historical_data.csv'
df2 = data_loader.read_csv_file(all_data_path)
df2 = df2.get("data")
df2['Date'] = pd.to_datetime(df2['Date'], format='ISO8601')

df2.head()
df2.shape

(6846, 9)

In [53]:
# Create a DataFrame with dates from the AMZN data
dates = df2.index
text_data = pd.DataFrame(dates, columns=['Date'])


In [54]:
# Assign a random headline to each date
text_data['Headline'] = np.random.choice(amzn_group['headline'], size=len(text_data))

# Display the first few rows of the updated DataFrame
text_data.head()

,Date,Headline
0,0,Amazon shares are trading lower after the comp...
1,1,"Aircraft Prices, E-Commerce Conditions Are Rip..."
2,2,Making Sense Of Why Consumers Are Switching Th...
3,3,Stocks That Hit 52-Week Highs On Thursday
4,4,"Amazon, Walmart Shares Unaffected Following Ea..."


In [55]:
def calculate_sentiment(text):
    return TextBlob(text).sentiment.polarity

# Applying the sentiment analysis function to your headlines
text_data['Sentiment'] = text_data['Headline'].apply(calculate_sentiment)

In [56]:
combined_data = text_data.set_index('Date').join(df2['Close'])
combined_data.head()
combined_data['Daily Returns'] = combined_data['Close'].pct_change()
combined_data.head()

,Headline,Sentiment,Close,Daily Returns
Date,,,,
0,Amazon shares are trading lower after the comp...,0.00,0.097917,NaN
1,"Aircraft Prices, E-Commerce Conditions Are Rip...",0.00,0.086458,-0.117028
2,Making Sense Of Why Consumers Are Switching Th...,0.00,0.085417,-0.012040
3,Stocks That Hit 52-Week Highs On Thursday,0.00,0.081771,-0.042685
4,"Amazon, Walmart Shares Unaffected Following Ea...",0.11,0.071354,-0.127392


In [57]:
combined_data.dropna(inplace=True)
correlation = combined_data['Sentiment'].corr(combined_data['Daily Returns'])
print("Correlation between sentiment and daily stock returns:", correlation)


Correlation between sentiment and daily stock returns: -0.011163180510481008


In [58]:
correlation = combined_data['Sentiment'].corr(combined_data['Close'])
print("Correlation between sentiment and closing:", correlation)

Correlation between sentiment and closing: -0.01931344638168807


# Calculate the correlation for GOOGLE


In [59]:
goog_group = grouped.get_group('GOOG')

In [60]:
all_data_path = '../data/yfinance_data/GOOG_historical_data.csv'
df3 = data_loader.read_csv_file(all_data_path)
df3 = df3.get("data")
df3['Date'] = pd.to_datetime(df2['Date'], format='ISO8601')

df3.head()
df3.shape

(5020, 9)

In [61]:
# Create a DataFrame with dates from the GOOGL data
dates = df3.index
text_data = pd.DataFrame(dates, columns=['Date'])


In [62]:
# Assign a random headline to each date
text_data['Headline'] = np.random.choice(goog_group['headline'], size=len(text_data))
# Display the first few rows of the updated DataFrame
text_data.head()

,Date,Headline
0,0,Analyst: Facebook Next To Face Antitrust Scrutiny
1,1,Google 'People Operations' Head Steps Down As ...
2,2,From Alphabet Earnings Conference Call: Sees C...
3,3,"Google Shares Unaffected Wed., Thurs. Followin..."
4,4,'Google in Middle of Fight Over 5G Spectrum' -...


In [63]:

# Applying the sentiment analysis function to your headlines
text_data['Sentiment'] = text_data['Headline'].apply(calculate_sentiment)

In [64]:
combined_data = text_data.set_index('Date').join(df3['Close'])
combined_data.head()
combined_data['Daily Returns'] = combined_data['Close'].pct_change()
combined_data.head()


,Headline,Sentiment,Close,Daily Returns
Date,,,,
0,Analyst: Facebook Next To Face Antitrust Scrutiny,0.000000,2.499133,NaN
1,Google 'People Operations' Head Steps Down As ...,-0.155556,2.697639,0.079430
2,From Alphabet Earnings Conference Call: Sees C...,0.187500,2.724787,0.010064
3,"Google Shares Unaffected Wed., Thurs. Followin...",-0.025000,2.611960,-0.041408
4,'Google in Middle of Fight Over 5G Spectrum' -...,0.000000,2.640104,0.010775


In [65]:

combined_data.dropna(inplace=True)
correlation = combined_data['Sentiment'].corr(combined_data['Daily Returns'])
print("Correlation between sentiment and daily stock returns:", correlation)

Correlation between sentiment and daily stock returns: 0.010646692193241991


In [66]:
correlation = combined_data['Sentiment'].corr(combined_data['Close'])
print("Correlation between sentiment and closing:", correlation)

Correlation between sentiment and closing: -0.020943266946366234


# Correlation on Facebook

In [68]:
meta_group = grouped.get_group('FB')

In [69]:

all_data_path = '../data/yfinance_data/AMZN_historical_data.csv'
df4 = data_loader.read_csv_file(all_data_path)
df4 = df4.get("data")
df4['Date'] = pd.to_datetime(df4['Date'], format='ISO8601')
df4.head()
df4.shape

(6846, 9)

In [70]:
# Create a DataFrame with dates from the AAPL data
dates = df4.index
text_data = pd.DataFrame(dates, columns=['Date'])


In [71]:
# Assign a random headline to each date
text_data['Headline'] = np.random.choice(meta_group['headline'], size=len(text_data))
# Display the first few rows of the updated DataFrame
text_data.head()

,Date,Headline
0,0,Facebook Rebrands Cryptocurrency Wallet Subsid...
1,1,Facebook Says Has Seen Weakening Ads Business ...
2,2,"JP Morgan Maintains Overweight on Facebook, Lo..."
3,3,Big Stocks Moving After Hours As Market Cheers...
4,4,YouTube Sued By Cryptocurrency Firm For Allege...


In [72]:

# Applying the sentiment analysis function to your headlines
text_data['Sentiment'] = text_data['Headline'].apply(calculate_sentiment)




In [73]:
combined_data = text_data.set_index('Date').join(df4['Close'])
combined_data.head()
combined_data['Daily Returns'] = combined_data['Close'].pct_change()
combined_data.head()

,Headline,Sentiment,Close,Daily Returns
Date,,,,
0,Facebook Rebrands Cryptocurrency Wallet Subsid...,0.00,0.097917,NaN
1,Facebook Says Has Seen Weakening Ads Business ...,0.00,0.086458,-0.117028
2,"JP Morgan Maintains Overweight on Facebook, Lo...",0.00,0.085417,-0.012040
3,Big Stocks Moving After Hours As Market Cheers...,0.00,0.081771,-0.042685
4,YouTube Sued By Cryptocurrency Firm For Allege...,-0.15,0.071354,-0.127392


In [74]:

combined_data.dropna(inplace=True)
correlation = combined_data['Sentiment'].corr(combined_data['Daily Returns'])
print("Correlation between sentiment and daily stock returns:", correlation)

Correlation between sentiment and daily stock returns: 0.007807123985507666


In [75]:
correlation = combined_data['Sentiment'].corr(combined_data['Close'])
print("Correlation between sentiment and closing:", correlation)

Correlation between sentiment and closing: -0.010793159916492793


# Correlation on Nvidia

In [76]:
nvda_group = grouped.get_group('NVDA')

In [77]:
all_data_path = '../data/yfinance_data/NVDA_historical_data.csv'
df5 = data_loader.read_csv_file(all_data_path)
df5 = df5.get("data")
df5['Date'] = pd.to_datetime(df5['Date'], format='ISO8601')
df5.head()
df5.shape

(6421, 9)

In [78]:

# Create a DataFrame with dates from the AAPL data
dates = df5.index
text_data = pd.DataFrame(dates, columns=['Date'])

In [79]:
# Assign a random headline to each date
text_data['Headline'] = np.random.choice(nvda_group['headline'], size=len(text_data))
# Display the first few rows of the updated DataFrame
text_data.head()


,Date,Headline
0,0,Shares of several semiconductor and microchip ...
1,1,4 Reasons Why 'Fortnite' Has Won Over Gamers
2,2,NVIDIA and Intellectual Ventures Partner to Ac...
3,3,"Nvidia, Baidu to Collaborate on Artificial Int..."
4,4,"Barclays Maintains Overweight on NVIDIA, Lower..."


In [80]:
# Applying the sentiment analysis function to your headlines
text_data['Sentiment'] = text_data['Headline'].apply(calculate_sentiment)

In [81]:
combined_data = text_data.set_index('Date').join(df2['Close'])
combined_data.head()
combined_data['Daily Returns'] = combined_data['Close'].pct_change()
combined_data.head()


,Headline,Sentiment,Close,Daily Returns
Date,,,,
0,Shares of several semiconductor and microchip ...,0.125,0.097917,NaN
1,4 Reasons Why 'Fortnite' Has Won Over Gamers,0.000,0.086458,-0.117028
2,NVIDIA and Intellectual Ventures Partner to Ac...,0.300,0.085417,-0.012040
3,"Nvidia, Baidu to Collaborate on Artificial Int...",-0.600,0.081771,-0.042685
4,"Barclays Maintains Overweight on NVIDIA, Lower...",0.000,0.071354,-0.127392


In [82]:
combined_data.dropna(inplace=True)
correlation = combined_data['Sentiment'].corr(combined_data['Daily Returns'])
print("Correlation between sentiment and daily stock returns:", correlation)

Correlation between sentiment and daily stock returns: 0.010504521357953083


In [83]:
correlation = combined_data['Sentiment'].corr(combined_data['Close'])
print("Correlation between sentiment and closing:", correlation)

Correlation between sentiment and closing: -0.008862929486741642


# Correlation on Tesla

In [88]:
tesla_group = grouped.get_group('TSLA')


In [89]:
all_data_path = '../data/yfinance_data/AMZN_historical_data.csv'
df6 = data_loader.read_csv_file(all_data_path)
df6 = df6.get("data")
df6['Date'] = pd.to_datetime(df6['Date'], format='ISO8601')
df6.head()
df6.shape

(6846, 9)

In [90]:
# Create a DataFrame with dates from the AAPL data
dates = df6.index
text_data = pd.DataFrame(dates, columns=['Date'])

In [92]:
# Assign a random headline to each date
text_data['Headline'] = np.random.choice(tesla_group['headline'], size=len(text_data))
# Display the first few rows of the updated DataFrame
text_data.head()


,Date,Headline
0,0,Tesla Gets $1.6B Loan From Chinese Banks For S...
1,1,Nio To Eliminate 21% Of Global Workforce As Q2...
2,2,'Lordstown Motors claims it's going to beat Te...
3,3,Tesla Building 'Canopy Covers' To Hide Its Sol...
4,4,China Association of Automobile Manufacturers ...


In [93]:
# Applying the sentiment analysis function to your headlines
text_data['Sentiment'] = text_data['Headline'].apply(calculate_sentiment)

In [94]:
combined_data = text_data.set_index('Date').join(df2['Close'])
combined_data.head()
combined_data['Daily Returns'] = combined_data['Close'].pct_change()
combined_data.head()


,Headline,Sentiment,Close,Daily Returns
Date,,,,
0,Tesla Gets $1.6B Loan From Chinese Banks For S...,0.000000,0.097917,NaN
1,Nio To Eliminate 21% Of Global Workforce As Q2...,0.000000,0.086458,-0.117028
2,'Lordstown Motors claims it's going to beat Te...,0.000000,0.085417,-0.012040
3,Tesla Building 'Canopy Covers' To Hide Its Sol...,0.000000,0.081771,-0.042685
4,China Association of Automobile Manufacturers ...,0.136364,0.071354,-0.127392


In [95]:
combined_data.dropna(inplace=True)
correlation = combined_data['Sentiment'].corr(combined_data['Daily Returns'])
print("Correlation between sentiment and daily stock returns:", correlation)

Correlation between sentiment and daily stock returns: 0.0017680702336444404


In [96]:

correlation = combined_data['Sentiment'].corr(combined_data['Close'])
print("Correlation between sentiment and closing:", correlation)


Correlation between sentiment and closing: -0.006510494936632368
